# Building Good Training Datasets - Data Preprocessing
Un modelo perfecto no sirve de nada si los dato que lo alimentan son de mala calidad. Por eso es de increible importancia entender y estudiar el proceso por el cual debe pasar un dataset antes de pasar por un algoritmo de ML. 

En este capitulo se discutiran las cuestiones escenciales de **pre procesamineto** por las cuales deberia pasar un dataset para poder construir buenos modelo.

Algunos de los  topicos que se veran en este capitulo son:
- Remover e imputar valores faltantes en un dataset
- Transformar informacion categorica en una forma que un algoritmo de ML pueda utilizar
- Feature engineer y feature seleccion

## Trabajar con informacion faltante.
Es normal que en la data de la vida real falte informacion en algunos campos por varias razones. Desde errores en la recoleccion de datos, errores en el input de los datos o incluso simplente que venga vacio es una posibilidad.
En general en los datasets estos valores apareceran como _NaN_, _Null_ o _None_. Sin embargo muchos algortimso no estan preprarados para manejar este tipo de data o las predicciones nacidas a partir de este tipo de informacion podria no ser confiable. En esta seccion veremos algunas estrategias para poder manejar este tipo de valores


### Identifying missin values in tabular data
Antes de discutir como manejar este tipo de datos, veamos un ejemplo sencillo de un DataFrame a partir de un CSV para visualizar el problema

In [2]:
import pandas as pd
from io import StringIO
csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [3]:
# En el caso mostrado es facil identificar los nulos,pero en dataframes mas grandes es directmente imposible.
# Por suerte podemos usar funciones para facilitar esto.
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

### Eliminating training examples or features with missing values
Una de las formas mas faciles de remover nulos es directamente removiendo la columna o fila por completo. Esta no es la mejor forma por razones obvias.

In [5]:
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [6]:
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [7]:
# Tambien hay un metodo que elimina solo si todas las columnas son NaN
df.dropna(how='all')


,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [8]:
# drop rows that have fewer than 4 real values
df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


### Imputing missing values
Muchas veces no se utiliza la opcion de droppear valores ya que podriamos terminar perdiendo mucha informacion valiosa. Por lo que otra alternativa muy comun es el de remplazas los valores faltantes por otro valor. 

Este metodo es muy delicado y hay varias maneras de hacerlo. Una de ellas es **mean imputation** donde remplazamos el valor faltante por el valor medio de la feature completa. Con skckit lear esto puede hacerse mediante la clase _SimpleImputer_

In [9]:
from sklearn.impute import SimpleImputer
import numpy as np
imr = SimpleImputer(missing_values=np.nan,strategy= 'mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

Otra estrategia muy comun es la de _most-frequent_, donde remplazamos el valor de nulo por el valor mas frecuente de la feture (mode). Este metodo es muy utilizado en features categoricas.

Otra forma directa es utilizar el metodo _fillna_ de pandas.

In [10]:
df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


## Handling categorical data
Hasta ahora solo trabajamos con valores numericos. Sin embargo es muy comun que en datos del mundo real tambien haya una o mas features categoricas. En esta seccion veremos algunas tecnicas simples para poder trabajar con este tipo de features.

Dentro de las features categoricas tenemos las **ordinales** y las **nominales**. Las ordianales son las features que poseen algun sentido de orden o jerarquia, por ejemplo los talles de ropa _XL_ > _L_ > _M_. Mientras que las nominales no poseen un orden asociado, como los colores.

### Categorical data encoding with pandas.
